Ejecutaremos un BATCH diarío a la hora del cierre de la tienda. En caso de tener servicio online lo realizaremos a las 2:00 AM para evitar horas con gran flujo de clientes.

# 5) Reporte desde parquet. / GOLD
ventas_file = OUT / "parquet" / "clean_ventas.parquet"
clientes_file = OUT / "parquet" / "clean_clientes.parquet"
productos_file = OUT / "parquet" / "clean_productos.parquet"

if ventas_file.exists():
    ventas = pd.read_parquet(ventas_file)
else:
    ventas = pd.DataFrame(columns=["fecha_venta","id_cliente","id_producto","unidades","precio_unitario","_ingest_ts"])

if clientes_file.exists():
    clientes = pd.read_parquet(clientes_file)
else:
    clientes = pd.DataFrame(columns=["id_cliente","nombre","apellido","fecha"])

if productos_file.exists():
    productos = pd.read_parquet(productos_file)
else:
    productos = pd.DataFrame(columns=["id_producto","nombre_producto","categoria","precio_unitario","unidades","fecha_entrada"])

if not ventas.empty:
    ventas["importe"] = ventas["unidades"].astype(float) * ventas["precio_unitario"].astype(float)

    # Left join para mostrar nombres de clientes y productos en ventas.
    ventas = ventas.merge(clientes[["id_cliente","nombre","apellido"]], on="id_cliente", how="left")
    ventas = ventas.merge(productos[["id_producto","nombre_producto","categoria"]], on="id_producto", how="left")

    ingresos = ventas["importe"].sum()
    trans = len(ventas)
    ticket = ingresos / trans if trans > 0 else 0.0

    top_prod = (ventas.groupby(["id_producto","nombre_producto","categoria"], as_index=False)
                    .agg(importe=("importe","sum"))
                    .sort_values("importe", ascending=False).head(5))
    total_imp = top_prod["importe"].sum() or 1.0
    top_prod["pct"] = (100 * top_prod["importe"] / total_imp).round(0).astype(int).astype(str) + "%"

    top_cli = (ventas.groupby("id_cliente", as_index=False)
                .agg(importe=("importe","sum"))
                .sort_values("importe", ascending=False)
                .head(5)
                .merge(clientes[["id_cliente","nombre","apellido"]], on="id_cliente", how="left"))

    by_day = (ventas.groupby("fecha_venta", as_index=False)
                    .agg(importe_total=("importe","sum"),
                        transacciones=("importe","count")).head(10))
    
    mayor_venta = ventas.loc[ventas["importe"].idxmax()]
    mayor_venta_info = (
        f"- Fecha: {mayor_venta['fecha_venta']}\n"
        f"- Cliente: {mayor_venta['nombre']} {mayor_venta['apellido']} (ID: {mayor_venta['id_cliente']})\n"
        f"- Producto: {mayor_venta['nombre_producto']} (ID: {mayor_venta['id_producto']})\n"
        f"- Unidades: {mayor_venta['unidades']}\n"
        f"- Importe: {mayor_venta['importe']:.2f} €") if not mayor_venta.empty else "—" 
    
    periodo_ini = str(ventas["fecha_venta"].min())
    periodo_fin = str(ventas["fecha_venta"].max())
    producto_lider = top_prod.iloc[0]["nombre_producto"] if not top_prod.empty else "—"
else:
    ingresos = 0.0; ticket = 0.0; trans = 0
    top_prod = pd.DataFrame(columns=["id_producto","nombre_producto","categoria","importe","pct"])
    top_cli = pd.DataFrame(columns=["id_cliente","nombre","apellido","importe"])
    by_day = pd.DataFrame(columns=["fecha_venta","importe_total","transacciones"])
    periodo_ini = "—"; periodo_fin = "—"; producto_lider = "—"

report = (
    "# Reporte UT1 · Ventas\n"
    f"**Periodo:** {periodo_ini} a {periodo_fin} · **Fuente:** clean_ventas (Parquet) · **Generado:** {datetime.now(timezone.utc).isoformat()}\n\n"
    "## 1. Titular\n"
    f"Ingresos totales {ingresos:.2f} €; producto líder: {producto_lider}.\n\n"
    "## 2. KPIs\n"
    f"- **Ingresos netos:** {ingresos:.2f} €\n"
    f"- **Ticket medio:** {ticket:.2f} €\n"
    f"- **Transacciones:** {trans}\n\n"
    "## 3. Top 5 productos\n"
    f"{(top_prod.to_markdown(index=False) if not top_prod.empty else '_(sin datos)_')}\n\n"
    "## 4. Top 5 clientes\n"
    f"{(top_cli.to_markdown(index=False) if not top_cli.empty else '_(sin datos)_')}\n\n"
    "## 5. Resumen por últimos 10 días\n"
    f"{(by_day.to_markdown(index=False) if not by_day.empty else '_(sin datos)_')}\n\n"  # <-- agregado aquí
    "## 6. Mayor venta\n"
    f"{mayor_venta_info}\n\n"
    "## 7. Persistencia\n"
    f"- Parquet: {ventas_file}, {clientes_file}, {productos_file}\n"
    f"- SQLite : {DB} (tablas: raw_*, clean_*; vistas: ventas_diarias)\n\n"
    "## 8. Conclusiones\n"
    "- Reponer producto líder según demanda.\n"
    "- Identificar clientes clave para fidelización.\n"
    "- Revisar filas en cuarentena (rangos/tipos).\n"
    "- Valorar particionado por fecha para crecer.\n"
)

(OUT / "reporte.md").write_text(report, encoding="utf-8")
print("✅ Reporte generado:", OUT / "reporte.md")